# Typeform: ML Case (3): PySpark

Run this notebook on an standalone spark on the root folder and feed it with the "typeform.csv" data file.

note: an error occurs while fitting the model.

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### Typeform: PySpark: Reading the dataframe


In [2]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType

In [3]:
df = spark.read.load("./typeform.csv", format="text")\
    .withColumn('form_id', f.split('value', ',')[0])\
    .withColumn('submissions', f.split('value', ',')[1].cast(IntegerType()))\
    .withColumn('views', f.split('value', ',')[2].cast(IntegerType()))\
    .withColumn('features', f.split('value', ',')[3])\
    .select('form_id','submissions','views','features')

In [4]:
df = df.rdd.map(lambda x : (
    int(x.form_id[1:]),
    x.submissions,
    x.views,
    x.submissions / x.views if x.views > 0.0 else 0.0,
    x.features[:-1]
)).toDF()\
    .withColumnRenamed("_1","form_id")\
    .withColumnRenamed("_2","submissions")\
    .withColumnRenamed("_3","views")\
    .withColumnRenamed("_4","completion_rate")\
    .withColumnRenamed("_5","features")

df.take(2)

[Row(form_id=1113027, submissions=33, views=27, completion_rate=1.2222222222222223, features='0.0-0.0-0.0-0.0-0.0-0.0-1.0-0.0-1.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-2.0-0.0-2.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-2.0-1.0-2.0'),
 Row(form_id=1115313, submissions=147, views=111, completion_rate=1.3243243243243243, features='0.0-2.0-0.0-0.0-0.0-0.0-0.0-0.0-1.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-3.0-0.0-3.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0-0.0')]

In [5]:
df.printSchema()

root
 |-- form_id: long (nullable = true)
 |-- submissions: long (nullable = true)
 |-- views: long (nullable = true)
 |-- completion_rate: double (nullable = true)
 |-- features: string (nullable = true)



### Typeform: PySpark: Features clean and unstack


In [6]:
df = df.where(df.views > 0.0)

In [8]:
features = [
   'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
   'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
   'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
   'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
   'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
   'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
   'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
   'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
   'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
   'feature_45', 'feature_46']

In [9]:
split_col = f.split(df['features'], '-')
for i in range(len(features)):
    df = df.withColumn(features[i], split_col.getItem(i).cast("double"))

In [10]:
df_ = df.select('completion_rate',
    'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
    'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
    'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
    'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
    'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
    'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
    'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
    'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
    'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
    'feature_45', 'feature_46')

### Typeform: PySpark: Features VectorAssembler


In [11]:
from pyspark.ml.feature import VectorAssembler

df_ = df_.where(df_.completion_rate.isNotNull())

assembler = VectorAssembler(
    inputCols=features,outputCol="features")

df_assembler = assembler.setHandleInvalid("skip").transform(df_)

In [13]:
df_train, df_test, df_validate = df_assembler.randomSplit([0.6, 0.35, 0.05])

### Typeform: PySpark: Model training linear regression

note: Py4JJavaError: An error occurred while calling o389.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 4.0 failed 1 times, most recent failure: Lost task 3.0 in stage 4.0 (TID 7, localhost, executor driver): java.lang.NoSuchMethodError: sun.nio.ch.DirectBuffer.cleaner()Lsun/misc/Cleaner;

In [17]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='completion_rate')
lr_model = lr.fit(c)

### Typeform: PySpark: Output statistics 

In [ ]:
trainingSummary = lr_model.summary

print("Coefficients: " + str(lr_model.coefficients))
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("R2: %f" % trainingSummary.r2)